Representing Qubit in code with Numpy

In [ ]:
# Representing Qubit in code with Numpy
import numpy as np
ket0 = np.array([[1],[0]])
ket1 = np.array([[0],[1]])

In [ ]:
ket0

array([[1],
       [0]])

In [ ]:
ket1

array([[0],
       [1]])

The Vector representation of |+》

In [ ]:
# The Vector representation of |+》
ket_plus = (ket0+ket1)/np.sqrt(2)
ket_plus

array([[0.70710678],
       [0.70710678]])

Defining the Hadamard operation

In [ ]:

# Defining the Hadamard operation
H = np.array([[1,1], [1,-1]]) / np.sqrt(2)

In [ ]:
H@ket0

array([[0.70710678],
       [0.70710678]])

In [ ]:
H@ket1

array([[ 0.70710678],
       [-0.70710678]])

Representing the Quantum NOT gate

In [ ]:
X = np.array([[0,1], [1,0]])

In [ ]:
X@ket0

array([[0],
       [1]])

In [ ]:
(X@ket0==ket1).all()

True

In [ ]:
X@H@ket0

array([[0.70710678],
       [0.70710678]])

We can use what we've learned so far to write the state of our qubit after each step in the QRNG algorithm

In [ ]:
# Pseudo code for a QRNG program
def qrng():
    q = Qubit()
    H(q)
    return measure(q)

Using matrix multiplication we can use a classical computer to simulate how `qrng()` would act on an ideal quantum device

First we require `QuantumDevice` class with abstract methods for allocating qubits, performing operations, and measuring qubits. We can then implement this class with a simulator and call into that simulator from `qrng()`

In [ ]:
from abc import ABCMeta, abstractmethod
from contextlib import contextmanager

class Qubit(metaclass=ABCMeta):
    @abstractmethod
    def h(self): pass

    @abstractmethod
    def measure(self) -> bool: pass

    @abstractmethod
    def reset(self): pass

class QuantumDevice(metaclass=ABCMeta):
    @abstractmethod
    def allocate_qubit(self) -> Qubit:
        pass

    @abstractmethod
    def deallocate_qubit(self, qubit: Qubit):
        pass

    @contextmanager
    def using_qubit(self):
        qubit = self.allocate_qubit()
        try:
            yield qubit
        finally:
            qubit.reset()
            self.deallocate_qubit(qubit)

With this in place, we can return to our definition of `qrng` using these new classes...

In [ ]:
def qrng(device: QuantumDevice) -> bool:
 with device.using_qubit() as q:
     q.h()
     return q.measure()

if __name__ == "__main__":
    qsim = SingleQubitSimulator()
    for idx_sample in range(10):
        random_sample = qrng(qsim)
        print(f"Our QRNG returned {random_sample}.")

... and if we implement the `QuantumDevice` interface with a class called `SingleQubitSimulator` , then we can pass this to `qrng` to run our QRNG implementation on a simulator.

Next, we define what a simulated qubit looks like. From the perspective of a simulator, a qubit wraps a vector that stores the qubit’s current state. We use a NumPy array to represent our qubit’s state.

In [ ]:

from interface import QuantumDevice, Qubit
import numpy as np

KET_0 = np.array([
    [1],
    [0]
], dtype=complex)

H = np.array([
    [1, 1],
    [1, -1]
], dtype=complex) / np.sqrt(2)

class SimulatedQubit(Qubit):
    def __init__(self):
        self.reset()
    def h(self):
        self.state = H @ self.state
    def measure(self) -> bool:
        pr0 = np.abs(self.state[0, 0]) ** 2
        sample = np.random.random() <= pr0
        return bool(0 if sample else 1)
    def reset(self):
        self.state = KET_0.copy()

class SingleQubitSimulator(QuantumDevice):
    available_qubits = [SimulatedQubit()]
    def allocate_qubit(self) -> SimulatedQubit:
        if self.available_qubits:
            return self.available_qubits.pop()
    def deallocate_qubit(self, qubit: SimulatedQubit):
        self.available_qubits.append(qubit)

Congratulations! We’ve not only written our first quantum program but also written a simulation backend and used it to run our quantum program the same way we’d run it on an actual quantum computer.

In [6]:
# Copyright (c) Sarah Kaiser and Chris Granade.
# Code sample from the book "Learn Quantum Computing with Python and Q#" by
# Sarah Kaiser and Chris Granade, published by Manning Publications Co.
# Book ISBN 9781617296130.
# Code licensed under the MIT License.